In [94]:
!pip install NLTK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [95]:
# Import necessary libraries
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
import pickle
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [96]:
# Load the dataset
df = pd.read_csv('data_bersih.csv')
df

,title,label,label_score
0,masuk radar pilwalkot medan menantu jokowi tem...,non-clickbait,0
1,malaysia sudut ri isu kabut asap invasi babi,non-clickbait,0
2,viral driver ojol bekas pesan makan pakai sepeda,clickbait,1
3,kemensos salur rp m korban rusuh sosial papua,non-clickbait,0
4,kait mayat bayi enas tangerang pria tangkap po...,non-clickbait,0
...,...,...,...
14995,tolak ruu tanah ribu tani gelar aksi istanadpr...,non-clickbait,0
14996,niat momong program hamil fedi nuril pengin ya...,clickbait,1
14997,edar isu internet papua blokir telkom warga pe...,non-clickbait,0
14998,txt comeback soobin aku gatal pamer spoiler,clickbait,1


In [97]:
print(df.isnull().sum())

title          0
label          0
label_score    0
dtype: int64


In [98]:
# Pisahkan atribut dan label
X = df['title']
y = df['label_score']

In [99]:
X

0        masuk radar pilwalkot medan menantu jokowi tem...
1             malaysia sudut ri isu kabut asap invasi babi
2         viral driver ojol bekas pesan makan pakai sepeda
3            kemensos salur rp m korban rusuh sosial papua
4        kait mayat bayi enas tangerang pria tangkap po...
                               ...                        
14995    tolak ruu tanah ribu tani gelar aksi istanadpr...
14996    niat momong program hamil fedi nuril pengin ya...
14997    edar isu internet papua blokir telkom warga pe...
14998          txt comeback soobin aku gatal pamer spoiler
14999    emmy awards peter dinklage cetak rekor berkat ...
Name: title, Length: 15000, dtype: object

In [100]:
y

0        0
1        0
2        1
3        0
4        0
        ..
14995    0
14996    1
14997    0
14998    1
14999    0
Name: label_score, Length: 15000, dtype: int64

In [101]:
# mengubah fitur menjadi vektor tf-idf
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(X)

# membagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# melatih model KNN
k = 9
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)

# melakukan prediksi pada data uji
y_pred = knn.predict(X_test)

# mengevaluasi model
accuracy = knn.score(X_test, y_test)
print('Akurasi:', accuracy)

Akurasi: 0.7146666666666667
